신경망 기본. 신경망 학습에 tensorflow를 사용하는데, tensorflow는 딥러닝에도 사용되므로 이 실습에서 공부한 내용은 모두 딥러닝에도 적용되는 것임.

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
# 필요한 모듈 임포트 및 필요한 설정
import numpy as np
import matplotlib.pyplot as plt
import sklearn  # scikit-learn 모듈 임포트
import os
import tensorflow as tf
from keras import models, layers
from keras.layers import Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator

# 랜덤하게 실행되는 부분들에 대해 다음에 실행해도 동일한 결과를 얻기 위해 설정
seed = 11
rng = np.random.default_rng(seed)
tf.random.set_seed(seed) # tensorflow에서 랜덤 요소 재현을 위해 seed 값 설정

# 분류용 다층 퍼셉트론

## 데이터 준비

In [39]:
base_dir = '/content/drive/My Drive/Colab Notebooks/Rice_Image_Dataset'

train_dir = os.path.join(base_dir, "train")
test_dir = os.path.join(base_dir, "test")
validation_dir = os.path.join(base_dir, "validation")

train_arborio_dir = os.path.join(train_dir, "Arborio")
train_basmati_dir = os.path.join(train_dir, "Basmati")
train_ipsala_dir = os.path.join(train_dir, "Ipsala")
train_Jasmine_dir = os.path.join(train_dir, "Jasmine")
train_karacadag_dir = os.path.join(train_dir, "Karacadag")

test_arborio_dir = os.path.join(test_dir, "Arborio")
test_basmati_dir = os.path.join(test_dir, "Basmati")
test_ipsala_dir = os.path.join(test_dir, "Ipsala")
test_Jasmine_dir = os.path.join(test_dir, "Jasmine")
test_karacadag_dir = os.path.join(test_dir, "Karacadag")

validation_arborio_dir = os.path.join(validation_dir, "Arborio")
validation_basmati_dir = os.path.join(validation_dir, "Basmati")
validation_ipsala_dir = os.path.join(validation_dir, "Ipsala")
validation_Jasmine_dir = os.path.join(validation_dir, "Jasmine")
validation_karacadag_dir = os.path.join(validation_dir, "Karacadag")

# 각 폴더에 있는 파일의 개수를 출력
print('훈련용 Arborio 데이터 개수: ', len(os.listdir(train_arborio_dir)))
print('훈련용 Basmati 데이터 개수: ', len(os.listdir(train_basmati_dir)))
print('훈련용 Ipsala 데이터 개수: ', len(os.listdir(train_ipsala_dir)))
print('훈련용 Jasmine 데이터 개수: ', len(os.listdir(train_Jasmine_dir)))
print('훈련용 Karacadag 데이터 개수: ', len(os.listdir(train_karacadag_dir)))

print('테스트용 Arborio 데이터 개수: ', len(os.listdir(test_arborio_dir)))
print('테스트용 Basmati 데이터 개수: ', len(os.listdir(test_basmati_dir)))
print('테스트용 Ipsala 데이터 개수: ', len(os.listdir(test_ipsala_dir)))
print('테스트용 Jasmine 데이터 개수: ', len(os.listdir(test_Jasmine_dir)))
print('테스트용 Karacadag 데이터 개수: ', len(os.listdir(test_karacadag_dir)))

print('검증용 Arborio 데이터 개수: ', len(os.listdir(validation_arborio_dir)))
print('검증용 Basmati 데이터 개수: ', len(os.listdir(validation_basmati_dir)))
print('검증용 Ipsala 데이터 개수: ', len(os.listdir(validation_ipsala_dir)))
print('검증용 Jasmine 데이터 개수: ', len(os.listdir(validation_Jasmine_dir)))
print('검증용 Karacadag 데이터 개수: ', len(os.listdir(validation_karacadag_dir)))

훈련용 Arborio 데이터 개수:  10354
훈련용 Basmati 데이터 개수:  10000
훈련용 Ipsala 데이터 개수:  10000
훈련용 Jasmine 데이터 개수:  10000
훈련용 Karacadag 데이터 개수:  10000
테스트용 Arborio 데이터 개수:  2500
테스트용 Basmati 데이터 개수:  2500
테스트용 Ipsala 데이터 개수:  4431
테스트용 Jasmine 데이터 개수:  4741
테스트용 Karacadag 데이터 개수:  4734
검증용 Arborio 데이터 개수:  2500
검증용 Basmati 데이터 개수:  2500
검증용 Ipsala 데이터 개수:  2500
검증용 Jasmine 데이터 개수:  2500
검증용 Karacadag 데이터 개수:  2500


In [40]:
train_datagen = ImageDataGenerator(rescale = 1./255, shear_range = 0.2,
  zoom_range = 0.2, horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)
validation_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,                      
    target_size=(250, 250), # 이미지 크기
    batch_size=20, # 배치: 한 번에 전처리할 수
    class_mode = 'categorical') # 다중 라벨링

test_generator = test_datagen.flow_from_directory(
    test_dir,                      
    target_size=(250, 250), 
    batch_size=20,
    class_mode = 'categorical')

validation_generator = validation_datagen.flow_from_directory(
    test_dir,                      
    target_size=(250, 250), 
    batch_size=20,
    class_mode = 'categorical')

Found 50354 images belonging to 5 classes.
Found 18906 images belonging to 5 classes.
Found 18906 images belonging to 5 classes.


In [41]:
print(train_generator.class_indices)
print(test_generator.class_indices)

{'Arborio': 0, 'Basmati': 1, 'Ipsala': 2, 'Jasmine': 3, 'Karacadag': 4}
{'Arborio': 0, 'Basmati': 1, 'Ipsala': 2, 'Jasmine': 3, 'Karacadag': 4}


## 모델 구조 생성
Sequential API를 사용해서 모델 구조를 생성함. Sequential 모델 생성 후 필요한 층을 입력층부터 순서대로 add함.

층 생성 시 뉴런 개수(`units`), 활성화 함수(`activation`) 등을 지정함. 선택적으로 층에 이름(`name`)을 부여할 수도 있음. 다양한 층들이 제공됨. https://www.tensorflow.org/api_docs/python/tf/keras/layers

이 중 MLP에 사용되는 층은 완전 연결 층인 Dense 층임. https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense

In [19]:
# 모델 생성
model = models.Sequential()

# 층 추가
# 첫 번째 층에는 input_shape에 샘플의 크기를 지정해야 함. 3은 컬러 이미지, 1은 흑백 이미지
model.add(layers.Conv2D(32,(3,3), activation='relu', input_shape=(250,250,3), name="input"))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Conv2D(64,(3,3), activation='relu'))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Flatten()) # Flatten은 입력 이미지를 1차원 배열(열 벡터)로 변환
# 은닉층 추가
# Dense 층(완전 연결 층)은 뉴런들과 입력 사이의 모든 연결 가중치를 포함하는 자신의 가중치 행렬을 관리함
model.add(layers.Dense(units=512, activation='relu', name="hidden1")) # 뉴런 512개, 활성화 함수: ReLU
model.add(layers.Dense(units=300, activation='relu', name="hidden2")) # 뉴런 300개, 활성화 함수: ReLU
model.add(layers.Dense(units=100, activation='relu', name="hidden3")) # 뉴런 100개, 활성화 함수: ReLU

# 출력층 추가
# 클래스마다 뉴런 하나씩 할당: 총 5개
# 배타적인 멀티 클래스 분류 문제이므로 소프트맥스 활성화 함수 사용
model.add(layers.Dense(units=5, activation='softmax', name="output"))

# 신경망 구조 출력
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (Conv2D)              (None, 248, 248, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 124, 124, 32)     0         
 )                                                               
                                                                 
 conv2d (Conv2D)             (None, 122, 122, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 61, 61, 64)       0         
 2D)                                                             
                                                                 
 flatten_5 (Flatten)         (None, 238144)            0         
                                                                 
 hidden1 (Dense)             (None, 512)              

In [20]:
model.layers[1].name

'max_pooling2d'

In [21]:
model.get_layer('hidden1')

In [22]:
# 연결 가중치와 편향 조회
weights, biases = model.layers[5].get_weights()

In [23]:
weights

array([[-1.2552147e-03, -4.0315101e-03,  8.9535955e-04, ...,
         3.3114087e-03,  1.3044239e-03, -4.2418018e-05],
       [-5.5351714e-04,  1.1704792e-03,  3.7190421e-03, ...,
         1.3731406e-03,  1.5620254e-03, -4.5658438e-03],
       [-4.5152288e-04, -4.8251967e-03, -1.6855688e-03, ...,
        -4.1507818e-03, -1.4274651e-03,  2.5113164e-03],
       ...,
       [-3.5710579e-03, -7.9700304e-04, -2.9796655e-03, ...,
        -2.9106950e-04,  2.7784253e-03,  3.2173470e-03],
       [ 4.6842443e-03,  3.8680062e-03, -4.8273476e-04, ...,
         3.5416298e-03,  2.2287513e-03,  4.9491180e-03],
       [ 2.9172096e-03,  4.5907907e-03, -2.3438269e-04, ...,
         7.3199254e-04,  3.3767382e-04, -4.4994140e-03]], dtype=float32)

In [42]:
weights.shape

(238144, 512)

In [43]:
biases

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [44]:
biases.shape

(512,)

## 모델 컴파일

생성된 모델의 `compile` 함수 사용. 주요 파라미터:
- `loss`: 손실 함수. tf.keras.losses에 정의된 손실 함수 이름 또는 tf.keras.losses.Loss 객체. https://www.tensorflow.org/api_docs/python/tf/keras/losses
 * 평균 제곱 오차: 'MSE' or 'mse' or 'mean_squared_error'
 * 이진 분류: 'binary_crossentropy' (출력층의 활성화 함수는 'sigmoid'로 설정)
 * 다중 분류(학습 집합의 레이블 값이 원-핫 벡터 형식이 아닌 정수 값인 경우): 'sparse_categorical_crossentropy' (출력층의 활성화 함수: 'softmax')
 * 다중 분류(학습 집합의 레이블 값이 원-핫 벡터 형식인 경우): 'categorical_crossentropy' (출력층의 활성화 함수: 'softmax')

- `optimizer`: 옵티마이저. tf.keras.optimizers에 정의된 옵티마이저 이름이나 옵티마이저 객체. https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
 * 기본적인 경사 하강법 옵티마이저(역전파 알고리즘 수행): 'sgd'

- `metrics`: 학습/테스트 과정에서 모델 평가에 사용할 평가 척도 리스트. tf.keras.metrics에 정의된 함수 이름 또는 tf.keras.metrics.Metric 객체. https://www.tensorflow.org/api_docs/python/tf/keras/metrics
 * 주로 `metrics=['accuracy']` 형태로 많이 사용됨. 'accuracy'나 'acc'로 지정될 경우, 손실 함수 및 출력 모양을 기반으로 BinaryAccuracy, CategoricalAccuracy, SparseCategoricalAccuracy 중 하나로 자동 변환됨. 즉, 정확도를 계산하게됨.

In [46]:
model.compile(optimizer = 'sgd', loss="categorical_crossentropy", metrics = ['accuracy'])

## 모델 학습

모델의 `fit` 함수 사용. 주요 파라미터:
- `epochs`: 학습 에포크 횟수. 기본값=1. 몇 번의 에포크 동안 학습할 것인가? 전체 학습 샘플을 학습에 이용했을 때 1 에포크라고 함. 에포크가 끝날 때마다 검증 집합을 사용해 손실과 metrics에서 지정한 평가 척도를 계산함.
- `batch_size`: 미니배치 크기. 기본값=32. 이 개수만큼의 샘플을 본 후 그레이디언트를 종합하여 경사 하강법을 한 번씩 적용(파리미터 업데이트 수행). 한 에포크 동안 (총 샘플 수) / (batch_size) 만큼의 업데이트(step)가 수행됨.
- `validation_data`: 각 에포크 후 손실과 metrics에서 지정한 평가 척도 계산에 사용할 검증 데이터. 모델은 이 데이터에 대해 학습을 수행하지는 않음.
- `validation_split`: 검증 데이터로 사용할 학습 데이터의 비율(0~1). 이 값이 주어지면 이 비율만큼 학습 데이터의 뒷쪽 부분을 미리 떼어두고 학습을 진행함.

In [47]:
history = model.fit(
    train_generator, steps_per_epoch = 20, epochs=5, batch_size=32,
    validation_data=test_generator, validation_steps=5)

Epoch 1/5


InvalidArgumentError: ignored

In [ ]:
history.params

{'verbose': 1, 'epochs': 10, 'steps': 100}

In [ ]:
# 검증 집합에 대한 손실과 측정한 평가 척도 값
history.history

{'loss': [0.9196843504905701,
  0.5082248449325562,
  0.44024673104286194,
  0.4156515300273895,
  0.38492703437805176,
  0.36763206124305725,
  0.35051900148391724,
  0.36818549036979675,
  0.34805333614349365,
  0.34275540709495544],
 'accuracy': [0.6050000190734863,
  0.7944999933242798,
  0.8190000057220459,
  0.8195000290870667,
  0.8355000019073486,
  0.843500018119812,
  0.8529999852180481,
  0.8335000276565552,
  0.8634999990463257,
  0.8615000247955322],
 'val_loss': [0.8526742458343506,
  1.7598717212677002,
  1.1500129699707031,
  0.7006350755691528,
  1.0859496593475342,
  1.2580633163452148,
  0.8635439872741699,
  1.5607293844223022,
  1.133224368095398,
  1.1213738918304443],
 'val_accuracy': [0.6000000238418579,
  0.7300000190734863,
  0.6800000071525574,
  0.7900000214576721,
  0.5899999737739563,
  0.6800000071525574,
  0.7900000214576721,
  0.6800000071525574,
  0.7099999785423279,
  0.75]}

In [ ]:
history.history.keys()

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])

## 모델 평가

모델의 `evaluate` 함수 사용. 손실 값과 metrics 평가 척도 값이 리스트 형태로 리턴됨. metrics 파라미터가 설정되지 않은 경우는 손실 값 하나만 리턴됨.

In [26]:
# 학습 곡선
import pandas as pd

pd.DataFrame(history.history).plot(figsize=(12, 7))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()

NameError: ignored

In [ ]:
# 테스트 집합으로 평가
model.evaluate(validation_generator)

946/946 [==============================] - 3477s 4s/step - loss: 1.3040 - accuracy: 0.7091


[1.304015040397644, 0.7091399431228638]

In [ ]:
# 평가 결과로 리턴된 값들의 척도 이름 확인
model.metrics_names

['loss', 'accuracy']

# 모델 저장과 복원

In [ ]:
# 모델 저장
# HDF5 포맷으로 저장: 모델 구조, 모든 층의 모델 파라미터, 모든 층의 하이퍼파라미터, 옵티마이저 등
model.save("rice_image_model.h5")

In [25]:
from keras.models import load_model

# 모델 복원(로딩)
model = load_model("/content/drive/My Drive/Colab Notebooks/rice_image_model.h5")

In [ ]:
# 테스트 집합으로 평가
model.evaluate(validation_generator)

351/946 [==========>...................] - ETA: 22:16 - loss: 0.8914 - accuracy: 0.7184

KeyboardInterrupt: ignored

# 체크포인트 및 조기 종료
학습 완료하는데 너무 많은 시간이 걸리면? 중간에 컴퓨터가 다운되면?
학습이 완료되기 전이라도 학습 도중 일정 간격으로 체크포인트 저장. fit() 메서드에서 콜백 이용.

In [ ]:
tf.keras.backend.clear_session() # Keras 상태 리셋

rng = np.random.default_rng(seed)
tf.random.set_seed(seed)

In [ ]:
model = models.Sequential([
    layers.Conv2D(32,(3,3), activation='relu', input_shape=(250,250,3), name="input"),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64,(3,3), activation='relu'),
    layers.MaxPooling2D(2,2), 
    layers.Flatten(), 
    layers.Dense(units=512, activation='relu', name="hidden1"), 
    layers.Dense(units=300, activation='relu', name="hidden2"), 
    layers.Dense(units=100, activation='relu', name="hidden3"), 
    layers.Dense(units=5, activation='softmax', name="output")
])

model.compile(optimizer = 'sgd', loss="categorical_crossentropy", metrics = ['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

# 체크포인트 콜백 정의: 검증 집합에서 이전 모델보다 성능이 더 좋을 때만 모델 저장
checkpoint_cb = ModelCheckpoint("rice_image_model.h5", save_best_only=True)

# 조기 종료 콜백 정의: 일정 에포크(patience) 동안 검증 집합에 대한 점수가 향상되지 않으면 학습 종료. 종료 후 최상의 모델로 복원.
early_stopping_cb = EarlyStopping(patience=5, restore_best_weights=True)

history = model.fit(
    train_generator, steps_per_epoch = 100, epochs=10, batch_size=32,
    validation_data=test_generator, validation_steps=5, callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/10
100/100 [==============================] - 363s 4s/step - loss: 1.0053 - accuracy: 0.5890 - val_loss: 1.6153 - val_accuracy: 0.4400
Epoch 2/10
100/100 [==============================] - 357s 4s/step - loss: 0.5116 - accuracy: 0.7865 - val_loss: 0.6928 - val_accuracy: 0.7700
Epoch 3/10
100/100 [==============================] - 342s 3s/step - loss: 0.4287 - accuracy: 0.8260 - val_loss: 1.3806 - val_accuracy: 0.5900
Epoch 4/10
100/100 [==============================] - 336s 3s/step - loss: 0.4155 - accuracy: 0.8210 - val_loss: 1.5339 - val_accuracy: 0.7300
Epoch 5/10
100/100 [==============================] - 341s 3s/step - loss: 0.4037 - accuracy: 0.8305 - val_loss: 1.1891 - val_accuracy: 0.6600
Epoch 6/10
100/100 [==============================] - 323s 3s/step - loss: 0.3731 - accuracy: 0.8490 - val_loss: 1.3947 - val_accuracy: 0.7000
Epoch 7/10
100/100 [==============================] - 319s 3s/step - loss: 0.3561 - accuracy: 0.8445 - val_loss: 1.3816 - val_accuracy: 0.7200

In [ ]:
# 테스트 집합으로 평가
model.evaluate(validation_generator) # 이 모델은 학습 과정에서 가장 좋은 성능을 보이는 모델로 복원된 결과임.

946/946 [==============================] - 581s 614ms/step - loss: 0.9788 - accuracy: 0.6904


[0.9787502288818359, 0.6904157400131226]

# 가중치 초기화 및 활성화 함수 개선
Dense 층의 가중치 초기화 기본값은 'glorot_uniform'(글로럿 균등분포 초기화)

In [ ]:
tf.keras.backend.clear_session() # Keras 상태 리셋

rng = np.random.default_rng(seed)
tf.random.set_seed(seed)

In [ ]:
# 은닉층이 100개인 심층 신경망
# ReLU, He 초기화

# 모델 생성
model = models.Sequential()

# 층 추가
# 첫 번째 층에는 input_shape에 샘플의 크기를 지정해야 함. 3은 컬러 이미지, 1은 흑백 이미지
model.add(layers.Conv2D(32,(3,3), activation='relu', input_shape=(250,250,3), name="input"))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Conv2D(64,(3,3), activation='relu'))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Flatten()) # Flatten은 입력 이미지를 1차원 배열(열 벡터)로 변환
# 은닉층 추가
# Dense 층(완전 연결 층)은 뉴런들과 입력 사이의 모든 연결 가중치를 포함하는 자신의 가중치 행렬을 관리함
model.add(layers.Dense(units=512, activation='relu', name="hidden1")) # 뉴런 512개, 활성화 함수: ReLU
model.add(layers.Dense(units=300, activation='relu', name="hidden2")) # 뉴런 300개, 활성화 함수: ReLU
model.add(layers.Dense(units=100, activation='relu', kernel_initializer="he_normal"))
for layer in range(99):
    model.add(Dense(100, activation="relu", kernel_initializer="he_normal"))

# 출력층 추가
# 클래스마다 뉴런 하나씩 할당: 총 5개
# 배타적인 멀티 클래스 분류 문제이므로 소프트맥스 활성화 함수 사용
model.add(layers.Dense(units=5, activation='softmax', name="output"))

In [ ]:
from tensorflow.keras.optimizers import SGD

model.compile(optimizer = SGD(learning_rate=0.001), loss="categorical_crossentropy", metrics = ['accuracy'])

In [ ]:
# 예제 코드와 다르게 epochs 값을 훨씬 크게 설정하고 체크포인트, 조기종료 등을 함께 사용함.

# 체크포인트 콜백 정의: 검증 집합에서 이전 모델보다 성능이 더 좋을 때만 모델 저장
checkpoint_cb = ModelCheckpoint("rice_image_model.h5", save_best_only=True)

# 조기 종료 콜백 정의: 일정 에포크(patience) 동안 검증 집합에 대한 점수가 향상되지 않으면 학습 종료. 종료 후 최상의 모델로 복원.
early_stopping_cb = EarlyStopping(patience=10, restore_best_weights=True)

#여기서 부터 실행 시간이 오래 걸리는 점을 고려해 steps_per_epoch = 5, epochs=10 로 조정함.
history = model.fit(
    train_generator, steps_per_epoch = 5, epochs=10, batch_size=32,
    validation_data=test_generator, validation_steps=5, callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/10
5/5 [==============================] - 323s 8s/step - loss: 0.3314 - accuracy: 0.8800 - val_loss: 0.7936 - val_accuracy: 0.6900
Epoch 2/10
5/5 [==============================] - 34s 7s/step - loss: 0.3856 - accuracy: 0.8500 - val_loss: 0.8581 - val_accuracy: 0.7800
Epoch 3/10
5/5 [==============================] - 37s 7s/step - loss: 0.3467 - accuracy: 0.8700 - val_loss: 1.3568 - val_accuracy: 0.6400
Epoch 4/10
5/5 [==============================] - 35s 7s/step - loss: 0.4146 - accuracy: 0.8100 - val_loss: 1.2505 - val_accuracy: 0.7400
Epoch 5/10
5/5 [==============================] - 35s 7s/step - loss: 0.3825 - accuracy: 0.8200 - val_loss: 0.8007 - val_accuracy: 0.7200
Epoch 6/10
5/5 [==============================] - 35s 7s/step - loss: 0.3583 - accuracy: 0.8600 - val_loss: 1.6080 - val_accuracy: 0.6800
Epoch 7/10
5/5 [==============================] - 33s 6s/step - loss: 0.2906 - accuracy: 0.8800 - val_loss: 0.9713 - val_accuracy: 0.7100
Epoch 8/10
5/5 [=================

In [ ]:
# 테스트 집합으로 평가
model.evaluate(validation_generator)

946/946 [==============================] - 587s 620ms/step - loss: 0.8771 - accuracy: 0.5628


[0.8771016597747803, 0.5627843141555786]

# 배치 정규화

In [29]:
tf.keras.backend.clear_session() # Keras 상태 리셋

rng = np.random.default_rng(seed)
tf.random.set_seed(seed)

In [30]:
#train_generator test_generator validation_generator 를 배치 정규화 부터는
#X_train, y_train , X_valid, y_valid, X_test, y_test 로 쓰기 위해 변수로 넘겨줌

X_train, y_train = train_generator.next()
X_valid, y_valid = validation_generator.next()
X_test, y_test = test_generator.next()

In [31]:
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import Sequential


# 첫번째 층과 각 은닉층 다음에 배치 정규화 층 적용
model = Sequential([
    Flatten(input_shape=X_train.shape[1:]),
    BatchNormalization(),
    Dense(300, activation="relu"),
    BatchNormalization(),
    Dense(100, activation="relu"),
    BatchNormalization(),
    Dense(10, activation="softmax")
])

In [50]:
from tensorflow.keras.optimizers import SGD

model.compile(loss="categorical_crossentropy",
              optimizer=SGD(learning_rate=0.001),
              metrics=["accuracy"])

In [51]:
# 예제 코드와 다르게 epochs 값을 훨씬 크게 설정하고 체크포인트, 조기종료 등을 함께 사용함.

# 체크포인트 콜백 정의: 검증 집합에서 이전 모델보다 성능이 더 좋을 때만 모델 저장
checkpoint_cb = ModelCheckpoint("rice_image_model.h5", save_best_only=True)

# 조기 종료 콜백 정의: 일정 에포크(patience) 동안 검증 집합에 대한 점수가 향상되지 않으면 학습 종료. 종료 후 최상의 모델로 복원.
early_stopping_cb = EarlyStopping(patience=10, restore_best_weights=True)

history = model.fit(
    train_generator, steps_per_epoch = 5, epochs=10, batch_size=32,
    validation_data=test_generator, validation_steps=5, callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/10


InvalidArgumentError: ignored

In [ ]:
# 테스트 집합으로 평가
model.evaluate(X_test, y_test)

1/1 [==============================] - 1s 664ms/step - loss: 1.3108 - accuracy: 0.2500


[1.3107846975326538, 0.25]

활성화 함수 전에 배치 정규화 층을 추가하려면 은닉층에서 활성화 함수를 지정하지 않고, 배치 정규화 층 뒤에 별도의 층으로 활성화 함수 층(`tf.keras.layers.Activation`)을 추가해야 함.

배치 정규화 층은 입력마다 이동 파라미터를 포함하기 떄문에 이전 층에 편향 파라미터를 생략할 수 있음(`use_bias=False`).

In [ ]:
tf.keras.backend.clear_session() # Keras 상태 리셋

rng = np.random.default_rng(seed)
tf.random.set_seed(seed)

In [ ]:
from tensorflow.keras.layers import Activation

model = Sequential([
    Flatten(input_shape=X_train.shape[1:]),
    BatchNormalization(),

    Dense(300, use_bias=False),
    BatchNormalization(),
    Activation("relu"),

    Dense(100, use_bias=False),
    BatchNormalization(),
    Activation("relu"),

    Dense(10, activation="softmax")
])

In [34]:
model.layers[2]

In [35]:
model.layers[2].get_weights()

[array([[-0.00281919,  0.0035714 , -0.00234441, ...,  0.00236495,
          0.00156091, -0.00466522],
        [-0.00218094, -0.00490079,  0.00436317, ..., -0.00544248,
         -0.00439407, -0.00315495],
        [-0.0002586 , -0.00262475, -0.00233972, ..., -0.00310031,
          0.00042094,  0.00017181],
        ...,
        [ 0.00556941,  0.00200249,  0.0008825 , ...,  0.00511774,
         -0.00202502, -0.00307652],
        [ 0.00462291, -0.00159249,  0.00114632, ..., -0.00357346,
          0.003167  ,  0.00499019],
        [-0.00188188,  0.00499732, -0.00197933, ..., -0.00295383,
          0.00132278, -0.00399264]], dtype=float32),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

In [ ]:
from tensorflow.keras.optimizers import SGD

model.compile(loss="categorical_crossentropy",
              optimizer=SGD(learning_rate=0.001),
              metrics=["accuracy"])
# 예제 코드와 다르게 epochs 값을 크게 설정하고 체크포인트, 조기종료 등을 함께 사용함.

history = model.fit(
    train_generator, steps_per_epoch = 5, epochs=10, batch_size=32,
    validation_data=test_generator, validation_steps=5, callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/10
5/5 [==============================] - 38s 7s/step - loss: 0.3356 - accuracy: 0.9000 - val_loss: 0.9828 - val_accuracy: 0.7300
Epoch 2/10
5/5 [==============================] - 35s 7s/step - loss: 0.3801 - accuracy: 0.8500 - val_loss: 1.0217 - val_accuracy: 0.7000
Epoch 3/10
5/5 [==============================] - 36s 7s/step - loss: 0.2601 - accuracy: 0.9200 - val_loss: 0.8642 - val_accuracy: 0.7400
Epoch 4/10
5/5 [==============================] - 34s 7s/step - loss: 0.3588 - accuracy: 0.8500 - val_loss: 1.1366 - val_accuracy: 0.6700
Epoch 5/10
5/5 [==============================] - 33s 6s/step - loss: 0.3351 - accuracy: 0.8800 - val_loss: 0.8978 - val_accuracy: 0.7600
Epoch 6/10
5/5 [==============================] - 33s 7s/step - loss: 0.3761 - accuracy: 0.8400 - val_loss: 0.8458 - val_accuracy: 0.7700
Epoch 7/10
5/5 [==============================] - 34s 7s/step - loss: 0.4275 - accuracy: 0.7800 - val_loss: 0.8183 - val_accuracy: 0.7800
Epoch 8/10
5/5 [==================

# 옵티마이저

In [48]:
model = Sequential([
    Flatten(input_shape=X_train.shape[1:]),
    Dense(300, activation="relu"),
    Dense(100, activation="relu"),
    Dense(10, activation="softmax")
])

In [49]:
from tensorflow.keras.optimizers import SGD, RMSprop, Adam, Nadam

# 모멘텀 최적화
model.compile(loss="categorical_crossentropy",
              optimizer=SGD(learning_rate=0.001, momentum=0.9),
              metrics=["accuracy"])

# 예제 코드와 다르게 epochs 값을 크게 설정하고 체크포인트, 조기종료 등을 함께 사용함.

history = model.fit(
    train_generator, steps_per_epoch = 5, epochs=10, batch_size=32,
    validation_data=test_generator, validation_steps=5, callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/10


InvalidArgumentError: ignored

In [ ]:
# NAG(네스테로프 가속 경사)
model.compile(loss="categorical_crossentropy",
              optimizer=SGD(learning_rate=0.001, momentum=0.9, nesterov=True),
              metrics=["accuracy"])

# 예제 코드와 다르게 epochs 값을 크게 설정하고 체크포인트, 조기종료 등을 함께 사용함.
history = model.fit(
    train_generator, steps_per_epoch = 5, epochs=10, batch_size=32,
    validation_data=test_generator, validation_steps=5, callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/10
5/5 [==============================] - 38s 7s/step - loss: 0.2883 - accuracy: 0.8900 - val_loss: 0.8381 - val_accuracy: 0.7700
Epoch 2/10
5/5 [==============================] - 32s 6s/step - loss: 0.3290 - accuracy: 0.8400 - val_loss: 1.1360 - val_accuracy: 0.7100
Epoch 3/10
5/5 [==============================] - 36s 7s/step - loss: 0.4243 - accuracy: 0.8200 - val_loss: 0.8312 - val_accuracy: 0.7000
Epoch 4/10
5/5 [==============================] - 31s 6s/step - loss: 0.2972 - accuracy: 0.8800 - val_loss: 1.1564 - val_accuracy: 0.6800
Epoch 5/10
5/5 [==============================] - 28s 6s/step - loss: 0.3007 - accuracy: 0.8900 - val_loss: 1.1389 - val_accuracy: 0.7400
Epoch 6/10
5/5 [==============================] - 29s 6s/step - loss: 0.3456 - accuracy: 0.8800 - val_loss: 1.7125 - val_accuracy: 0.6600
Epoch 7/10
5/5 [==============================] - 30s 6s/step - loss: 0.3161 - accuracy: 0.8600 - val_loss: 1.2618 - val_accuracy: 0.7200
Epoch 8/10
5/5 [==================

In [ ]:
# RMSProp
model.compile(loss="categorical_crossentropy",
              optimizer=RMSprop(learning_rate=0.001, rho=0.9), # rho: 베타 값
              metrics=["accuracy"])

# 예제 코드와 다르게 epochs 값을 크게 설정하고 체크포인트, 조기종료 등을 함께 사용함.
history = model.fit(
    train_generator, steps_per_epoch = 5, epochs=10, batch_size=32,
    validation_data=test_generator, validation_steps=5, callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/10
5/5 [==============================] - 34s 5s/step - loss: 34.6208 - accuracy: 0.3000 - val_loss: 2.4712 - val_accuracy: 0.2900
Epoch 2/10
5/5 [==============================] - 30s 6s/step - loss: 2.5933 - accuracy: 0.3000 - val_loss: 1.8812 - val_accuracy: 0.2500
Epoch 3/10
5/5 [==============================] - 32s 6s/step - loss: 1.5135 - accuracy: 0.3800 - val_loss: 1.1912 - val_accuracy: 0.6000
Epoch 4/10
5/5 [==============================] - 31s 6s/step - loss: 1.0400 - accuracy: 0.5900 - val_loss: 1.4948 - val_accuracy: 0.5100
Epoch 5/10
5/5 [==============================] - 31s 6s/step - loss: 1.1433 - accuracy: 0.5600 - val_loss: 1.1969 - val_accuracy: 0.4800
Epoch 6/10
5/5 [==============================] - 30s 6s/step - loss: 1.1163 - accuracy: 0.5000 - val_loss: 1.4731 - val_accuracy: 0.5800
Epoch 7/10
5/5 [==============================] - 31s 6s/step - loss: 0.8715 - accuracy: 0.6200 - val_loss: 1.2191 - val_accuracy: 0.4200
Epoch 8/10
5/5 [=================

In [ ]:
# Adam
model.compile(loss="categorical_crossentropy",
              optimizer=Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999),
              metrics=["accuracy"])

# 예제 코드와 다르게 epochs 값을 크게 설정하고 체크포인트, 조기종료 등을 함께 사용함.
history = model.fit(
    train_generator, steps_per_epoch = 5, epochs=10, batch_size=32,
    validation_data=test_generator, validation_steps=5, callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/10
5/5 [==============================] - 31s 6s/step - loss: 0.7142 - accuracy: 0.6800 - val_loss: 1.4551 - val_accuracy: 0.7000
Epoch 2/10
5/5 [==============================] - 32s 6s/step - loss: 0.6476 - accuracy: 0.7300 - val_loss: 1.5932 - val_accuracy: 0.5900
Epoch 3/10
5/5 [==============================] - 29s 6s/step - loss: 0.6392 - accuracy: 0.6900 - val_loss: 1.0191 - val_accuracy: 0.6400
Epoch 4/10
5/5 [==============================] - 31s 6s/step - loss: 0.5086 - accuracy: 0.7300 - val_loss: 1.2702 - val_accuracy: 0.7000
Epoch 5/10
5/5 [==============================] - 29s 6s/step - loss: 0.3860 - accuracy: 0.8100 - val_loss: 1.4111 - val_accuracy: 0.6900
Epoch 6/10
5/5 [==============================] - 30s 6s/step - loss: 0.5457 - accuracy: 0.7500 - val_loss: 1.4757 - val_accuracy: 0.7000
Epoch 7/10
5/5 [==============================] - 33s 6s/step - loss: 0.5404 - accuracy: 0.7700 - val_loss: 1.1438 - val_accuracy: 0.7400
Epoch 8/10
5/5 [==================

In [9]:
# Nadam
model.compile(loss="categorical_crossentropy",
              optimizer=Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999),
              metrics=["accuracy"])

# 예제 코드와 다르게 epochs 값을 크게 설정하고 체크포인트, 조기종료 등을 함께 사용함.
history = model.fit(
    train_generator, steps_per_epoch = 5, epochs=10, batch_size=32,
    validation_data=test_generator, validation_steps=5, callbacks=[checkpoint_cb, early_stopping_cb])

NameError: ignored

# 드롭아웃

In [12]:
from tensorflow.keras.layers import Dropout
from keras.callbacks import ModelCheckpoint, EarlyStopping

# 체크포인트 콜백 정의: 검증 집합에서 이전 모델보다 성능이 더 좋을 때만 모델 저장
checkpoint_cb = ModelCheckpoint("rice_image_model.h5", save_best_only=True)

# 조기 종료 콜백 정의: 일정 에포크(patience) 동안 검증 집합에 대한 점수가 향상되지 않으면 학습 종료. 종료 후 최상의 모델로 복원.
early_stopping_cb = EarlyStopping(patience=5, restore_best_weights=True)

model = Sequential([
    Flatten(input_shape=X_train.shape[1:]),
    Dropout(rate=0.2),
    Dense(300, activation="elu", kernel_initializer="he_normal"),
    Dropout(rate=0.2),
    Dense(100, activation="elu", kernel_initializer="he_normal"),
    Dropout(rate=0.2),
    Dense(10, activation="softmax")
])

model.compile(loss="categorical_crossentropy", 
              optimizer="nadam", 
              metrics=["accuracy"])


# 예제 코드와 다르게 epochs 값을 크게 설정하고 체크포인트, 조기종료 등을 함께 사용함.
history = model.fit(
    train_generator, steps_per_epoch = 5, epochs=10, batch_size=32,
    validation_data=test_generator, validation_steps=5, callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/10


InvalidArgumentError: ignored

# 종합

In [ ]:
# 은닉층이 100개인 심층 신경망
# ELU, He 초기화, 배치 정규화
model = Sequential()
model.add(Flatten(input_shape=X_train.shape[1:]))
model.add(BatchNormalization())

model.add(Dense(300, kernel_initializer="he_normal", use_bias=False))
model.add(BatchNormalization())
model.add(Activation("elu"))

for layer in range(96):
    model.add(Dense(100, kernel_initializer="he_normal", use_bias=False))
    model.add(BatchNormalization())
    model.add(Activation("elu"))

for layer in range(3):
    model.add(Dense(100, kernel_initializer="he_normal", use_bias=False))
    model.add(BatchNormalization())
    model.add(Activation("elu"))
    Dropout(rate=0.5) # 드롭아웃
    
model.add(Dense(10, activation="softmax"))

In [ ]:
model.compile(loss="categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])

# 예제 코드와 다르게 epochs 값을 크게 설정하고 체크포인트, 조기종료 등을 함께 사용함.

history = model.fit(
    train_generator, steps_per_epoch = 5, epochs=10, batch_size=32,
    validation_data=test_generator, validation_steps=5, callbacks=[checkpoint_cb, early_stopping_cb])

In [ ]:
# 테스트 집합으로 평가
model.evaluate(X_test, y_test)